In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go


raw_data_path = '/Users/lubovmoskalenko/Documents/python/-pabd25/data/row/'
file_list = glob.glob(os.path.join(raw_data_path, "*.csv"))

dataframes = []
for file in file_list:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Ошибка при чтении {file}: {e}")

main_dataframe = pd.concat(dataframes, ignore_index=True)

main_dataframe['url_id'] = main_dataframe['url'].map(lambda x: x.split('/')[-2] if isinstance(x, str) else None)
df = main_dataframe[['url_id', 'total_meters','floor','floors_count','rooms_count', 'price']].dropna().set_index('url_id')

df = df[(df['total_meters'] <= 300) & (df['price'] < 100_000_000)]

df.to_csv('/Users/lubovmoskalenko/Documents/python/-pabd25/data/processed/merged_cleaned.csv', encoding='utf-8')

In [9]:
x = df['total_meters']
y = df['price']

m, b = np.polyfit(x, y, 1)
regression_line = m * x + b

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Квартиры'))
fig1.add_trace(go.Scatter(x=x, y=regression_line, mode='lines', name='Линейная регрессия', line=dict(color='red')))
fig1.update_layout(
    title='Цена от площади квартиры',
    xaxis_title='Площадь (м²)',
    yaxis_title='Цена (₽)',
    template='plotly_white'
)
fig1.show()

fig2 = px.histogram(df, x='price', nbins=50, title='Распределение цен на квартиры')
fig2.update_layout(
    xaxis_title='Цена (₽)',
    yaxis_title='Количество квартир',
    template='plotly_white'
)
fig2.show()